## Importing libraries

In [3]:
import tensorflow as tf

In [4]:
tf.config.list_physical_devices('GPU')

[]

In [5]:
import numpy as np # linear algebra
import pandas as pd

In [6]:

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
import cv2
import glob
import matplotlib as plt
import tensorflow.keras.layers as layers

## Loading data

In [8]:
os.listdir('../Dataset')

['cardboard',
 'glass',
 'metal',
 'nothing',
 'organic',
 'paper',
 'plastic',
 'README.md',
 'trash']

In [9]:
#reading an image
img = cv2.imread(os.path.join('../Dataset','nothing','0.jpg'))
print(img.shape)

(224, 224, 3)


In [10]:
nothing = glob.glob(os.path.join('dataset-resized','nothing','*.jpg'))
cardboard = glob.glob(os.path.join('dataset-resized','cardboard','*.jpg'))
glass = glob.glob(os.path.join('dataset-resized','glass','*.jpg'))
metal = glob.glob(os.path.join('dataset-resized','metal','*.jpg'))
organic = glob.glob(os.path.join('dataset-resized','organic','*.jpg'))
paper = glob.glob(os.path.join('dataset-resized','paper','*.jpg'))
plastic = glob.glob(os.path.join('dataset-resized','plastic','*.jpg'))
trash = glob.glob(os.path.join('dataset-resized','trash','*.jpg'))

In [8]:
from tqdm import tqdm

In [9]:
data = []
labels = []
for i in tqdm(nothing):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(0)
    
for i in tqdm(cardboard):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(1)

for i in tqdm(glass):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(2)
    
for i in tqdm(metal):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(3)

for i in tqdm(organic):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(4)
    
for i in tqdm(paper):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(5)
    
for i in tqdm(plastic):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(6)
    
for i in tqdm(trash):   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (384,512))
    image=np.array(image)
    data.append(image)
    labels.append(7)
    
data = np.array(data)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, ytrain, ytest = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)

100%|██████████| 137/137 [00:00<00:00, 334.15it/s]


## Creating an encoder

In [10]:
def create_encoder():
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),input_shape=(384,512,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
        
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu')])
    return model


encoder = create_encoder()
print(encoder.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 382, 510, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 191, 255, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 189, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 94, 126, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 92, 124, 128)      73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 46, 62, 128)      0

## Creating the classifier

In [11]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=(384,512,3))
    features = encoder(inputs)
    features = layers.Dropout(0.1)(features)
    features = layers.Dense(512, activation="relu")(features)
    features = layers.Dropout(0.1)(features)
    outputs = layers.Dense(8, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

### 1:Training the complete encoder+classifier

In [12]:
print(tf.test.is_built_with_cuda())

False


In [13]:
#following script might take very long time to execute on cpu

In [14]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

history=classifier.fit(X_train, ytrain,validation_split=0.2, epochs=5)

Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384, 512, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 256)               93547840  
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 93,683,528
Trainable params: 93,683,528
No

### 2: Freezing the classifier and training the encoder so that encoder learns to predict the most optimal image embeddings

In [15]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)

def add_projection_head(encoder):
    inputs = keras.Input(shape=(384,512,3))
    features = encoder(inputs)
    outputs = layers.Dense(256, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="encoder_with_projection-head"
    )
    return model

In [16]:
!pip install tensorflow-addons

In [17]:
pip install -U typing-extensions


Note: you may need to restart the kernel to use updated packages.


In [18]:
import tensorflow_addons as tfa

In [ ]:
learning_rate=0.001
temperature=0.05
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(X_train, ytrain,epochs=20)

Model: "encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 384, 512, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 256)               93547840  
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
Total params: 93,613,632
Trainable params: 93,613,632
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
112/112 [==============================] - 682s 6s/step - loss: 3.2870
Epoch 2/20
112/112 [==============================] - 699s 6s/step - loss: 2.8273
Epoch 3/20
112/112 [==============================] - 681s 6s/step - loss: 2.7103
Epoch 4/20
112/112 [========================

### 3: Training the classifier with frozen encoder

In [ ]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=X_train, y=ytrain, validation_split=0.2,batch_size=16, epochs=1)

## Plotting performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

## Save the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','classifierModel.h5'))